In [187]:
import pandas as pd
import numpy as np
import networkx as nx
import ast
import matplotlib.pyplot as plt
from datetime import datetime as dt
from random import randint
from collections import Counter
import math
import sklearn as sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.metrics.pairwise import linear_kernel

In [85]:
users = pd.read_csv('2018_02_20_Users.csv', encoding = "ISO-8859-1")

In [86]:
users.head(n=5)

,userHash (S),currentLocation (S),education1location (S),education1name (S),education2location (S),education2name (S),education3location (S),education3name (S),gender (S),hometown (S),...,work1role (S),work2location (S),work2name (S),work2role (S),work3location (S),work3name (S),work3role (S),work4location (S),work4name (S),work4role (S)
0,-318129462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,635926102,NaN,"Redmond, Washington",Redmond High School,"Redmond, Washington",University of Washington,NaN,NaN,Male,NaN,...,Software Developer,"San Jose, CA",Cisco,Software Engineering Intern,"Seattle, WA",Seattle Youth Symphony Orchestra,Data entry clerk,NaN,NaN,NaN
2,-1748993256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1318276747,Princeton,NaN,NaN,NaN,NaN,NaN,NaN,Female,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2109566321,NaN,"Redmond, WA",Redmond High School,"Redmond, WA",Princeton University,NaN,NaN,Female,"Sammamish, WA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
articles = pd.read_csv('2018_02_20_Articles.csv', encoding = "ISO-8859-1")

In [88]:
articles.head(n=5)

,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
0,-318129462,http://bit.ly/Timoth%C3%A9e-Armie-YOU,True,"Sun, 11 Feb 2018 02:44:44 GMT",[],[],"[ { ""S"" : ""Sun, 11 Feb 2018 02:44:44 GMT"" }]",[],NaN,[]
1,-318129462,http://eonli.ne/2BZ0vBf,True,"Fri, 16 Feb 2018 17:28:00 GMT",[],[],"[ { ""S"" : ""Fri, 16 Feb 2018 17:28:00 GMT"" }]",[],NaN,[]
2,-318129462,http://www.eonline.com/news/911768/jamie-dorna...,True,"Fri, 16 Feb 2018 17:28:08 GMT",[],[],"[ { ""S"" : ""Fri, 16 Feb 2018 17:28:08 GMT"" }]",[],NaN,[]
3,-318129462,https://offbeat.topix.net/slideshow/18943,True,"Sun, 11 Feb 2018 02:52:14 GMT",[],[],"[ { ""S"" : ""Sun, 11 Feb 2018 02:52:14 GMT"" }]",[],NaN,[]
4,-318129462,https://rewire.news/article/2018/02/14/south-a...,True,"Fri, 16 Feb 2018 17:18:31 GMT",[],[],"[ { ""S"" : ""Fri, 16 Feb 2018 17:18:29 GMT"" }]",[],NaN,[]


In [89]:
# take interactions that have sharerHashes
forgraph_t = articles[articles['sharerHash (L)'] != '[]']
forgraph_t = forgraph_t.reset_index()

In [107]:
forgraph_t.head(n=5)

,index,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
0,14,-1949465643,http://jstreet.org/press-releases/j-street-opp...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""10"" }]","[ { ""S"" : ""Tue, 13 Feb 2018 01:21:46 GMT"" }]","[ { ""S"" : ""02/12/2018 6:54pm"" }]",J Street Opposes Confirmation of Kenneth Marcu...,"[ { ""S"" : ""-1174757635"" }]"
1,19,-1748993256,http://hill.cm/EOrPLMZ,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""3 "" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:06:00 GMT"" }]","[ { ""S"" : ""NULL"" }]",White House refused to release photo of Trump ...,"[ { ""S"" : ""2407815"" }]"
2,20,-1748993256,http://hill.cm/X9HpS96,False,NaN,"[ { ""S"" : ""BREAKING: FBI officials were alert...","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/15/2018 8:05am"" }]",FBI was warned last year of school shooting th...,"[ { ""S"" : ""1101628624"" }]"
3,21,-1748993256,http://stanford.io/2ClZ68d,True,"Thu, 15 Feb 2018 14:32:01 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""44"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:32:01 GMT"" }]","[ { ""S"" : ""02/15/2018 8:45am"" }]","http://stanford.io/2ClZ68d</a> <br> <a class=""...","[ { ""S"" : ""46193696"" }]"
4,22,-1748993256,http://t.ted.com/gmdx0iv,False,NaN,"[ { ""S"" : ""Practical money advice from a self...","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 10:00am"" }]",6 simple tips for investment beginners,"[ { ""S"" : ""82931"" }]"


In [108]:
# take interactions that have userHashes that are in the users table
users_list = set(users["userHash (S)"])
forgraph = forgraph_t[forgraph_t["userHash (S)"].isin(users_list)] 

In [109]:
len(forgraph)

1341

In [110]:
forgraph.head(n=5)

,index,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
1,19,-1748993256,http://hill.cm/EOrPLMZ,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""3 "" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:06:00 GMT"" }]","[ { ""S"" : ""NULL"" }]",White House refused to release photo of Trump ...,"[ { ""S"" : ""2407815"" }]"
2,20,-1748993256,http://hill.cm/X9HpS96,False,NaN,"[ { ""S"" : ""BREAKING: FBI officials were alert...","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/15/2018 8:05am"" }]",FBI was warned last year of school shooting th...,"[ { ""S"" : ""1101628624"" }]"
3,21,-1748993256,http://stanford.io/2ClZ68d,True,"Thu, 15 Feb 2018 14:32:01 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""44"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:32:01 GMT"" }]","[ { ""S"" : ""02/15/2018 8:45am"" }]","http://stanford.io/2ClZ68d</a> <br> <a class=""...","[ { ""S"" : ""46193696"" }]"
4,22,-1748993256,http://t.ted.com/gmdx0iv,False,NaN,"[ { ""S"" : ""Practical money advice from a self...","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 10:00am"" }]",6 simple tips for investment beginners,"[ { ""S"" : ""82931"" }]"
5,23,-1748993256,https://collegepulse.com/survey/5a85e6d386f8d3...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 9:40am"" }]",www.collegepulse.com</a>,"[ { ""S"" : ""1236633202"" }]"


## Base methods

In [5]:
def getRandomInteraction(all_interactions):
    index = randint(0, len(all_interactions)-1)
    currUserHash = all_interactions.iloc[[index]]['userHash (S)'][index]
    currSharerHash_list = ast.literal_eval(all_interactions.iloc[[index]]['sharerHash (L)'][index])
    currSeenTime_list = ast.literal_eval(all_interactions.iloc[[index]]['articleSeenTime (L)'][index])
    if len(currSharerHash_list) == 1:
        currSharerHash = currSharerHash_list[0].get("S")
        currSeenTime = currSeenTime_list[0].get("S")
    else:
        t_index = randint(0, len(currSharerHash_list)-1)
        currSharerHash = currSharerHash_list[t_index].get("S")
        currSeenTime= currSeenTime_list[t_index].get("S")
    return [str(currUserHash), str(currSharerHash), str(currSeenTime)]

In [6]:
def makeGraph(timestamp):
    timestamp = dt.strptime(timestamp, "%a, %d %b %Y %H:%M:%S %Z")
    beforetime_DF = pd.DataFrame()
    for i in range(0, len(forgraph)):
        temp_date_list = ast.literal_eval(forgraph.iloc[[i]]["articleSeenTime (L)"][i])
        for k in range(0, len(temp_date_list)):
            temp_date = temp_date_list[k].get("S")
            date_obj = dt.strptime(temp_date, "%a, %d %b %Y %H:%M:%S %Z")
            if date_obj <= timestamp:
                beforetime_DF = beforetime_DF.append(forgraph.iloc[[i]], ignore_index=True)
                break
                
    sharerHash_list = beforetime_DF['sharerHash (L)']
    userHash_list = beforetime_DF["userHash (S)"]
    
    edges_list = []
    for i in range(0, len(sharerHash_list)):
        temp = ast.literal_eval(sharerHash_list[i])
        for k in range(0, len(temp)):
            edges_list.append((str(userHash_list[i]), temp[k].get("S")))
            
    weighted_edges = []
    i = 0
    while i < len(edges_list)-2:
        weight = 1
        while ((edges_list[i][0] == edges_list[i+1][0]) and (edges_list[i][1] == edges_list[i+1][1])):
            weight = weight + 1
            i = i + 1
        weighted_edges.append((edges_list[i][0], edges_list[i][1], weight))
        i = i + 1
        
    interactions = nx.DiGraph()
    interactions.add_weighted_edges_from(weighted_edges)
    return interactions

## Feature functions

In [ ]:
def thematicInterest():

In [7]:
def activity(x, g):
    return g.out_degree(x)/g.in_degree(x)

In [ ]:
def socialPressure():

## tf-idf

In [170]:
def tokenize(data):
    translator = str.maketrans('', '', string.punctuation)
    data = data.lower().translate(translator) 
    tokens = nltk.word_tokenize(data)
    return tokens

def removeStopwords(tokens):
    filtered_tokens = [w for w in tokens if not w in stopwords.words('english')]
    return filtered_tokens

def stem(tokens):
    stemmer = PorterStemmer()
    stemmed_tokens = []
    for item in tokens:
        stemmed_tokens.append(stemmer.stem(item))
    return stemmed_tokens

In [190]:
def getArticleData(ind):
    comments = ast.literal_eval(temp["articleComments (L)"][ind])[0].get("S")
    title = temp["articleTitle (S)"][ind]
    article_data = comments + " " + title
    return article_data

def getUserData(userHash): #user_info is a df that is a single row
    user_info = users[users["userHash (S)"] == userHash].reset_index()
    u_data = ""
    if str(user_info["currentLocation (S)"][0]) != "nan":
        u_data = u_data + user_info["currentLocation (S)"][0] + " "
    if str(user_info["education1location (S)"][0]) != "nan":
        u_data = u_data + user_info["education1location (S)"][0] + " "
    if str(user_info["education1name (S)"][0]) != "nan":
        u_data = u_data + user_info["education1name (S)"][0] + " "
    if str(user_info["education2location (S)"][0]) != "NaN":
        u_data = u_data + user_info["education2location (S)"][0] + " "
    if str(user_info["education2name (S)"][0]) != "NaN":
        u_data = u_data + user_info["education2name (S)"][0] + " "
    if str(user_info["education3location (S)"][0]) != "NaN":
        u_data = u_data + user_info["education3location (S)"][0] + " "
    if str(user_info["education3name (S)"][0]) != "NaN":
        u_data = u_data + user_info["education3name (S)"][0] + " "
    if str(user_info["gender (S)"][0]) != "NaN":
        u_data = u_data + user_info["gender (S)"][0] + " "
    if str(user_info["hometown (S)"][0]) != "NaN":
        u_data = u_data + user_info["hometown (S)"][0] + " "
    if str(user_info["language1 (S)"][0]) != "nan":
        u_data = u_data + str(user_info["language1 (S)"][0]) + " "
    if str(user_info["language2 (S)"][0]) != "nan":
        u_data = u_data + str(user_info["language2 (S)"][0]) + " "
    if str(user_info["language3 (S)"][0]) != "nan":
        u_data = u_data + str(user_info["language3 (S)"][0]) + " "
    if str(user_info["otherLocation1 (S)"][0]) != "nan":
        u_data = u_data + str(user_info["otherLocation1 (S)"][0]) + " "
    if str(user_info["otherLocation2 (S)"][0]) != "nan":
        u_data = u_data + str(user_info["otherLocation2 (S)"][0]) + " "
    if str(user_info["work1location (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work1location (S)"][0]) + " "
    if str(user_info["work1name (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work1name (S)"][0]) + " "
    if str(user_info["work1role (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work1role (S)"][0]) + " "
    if str(user_info["work2location (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work2location (S)"][0]) + " "
    if str(user_info["work2name (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work2name (S)"][0]) + " "
    if str(user_info["work2role (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work2role (S)"][0]) + " "
    if str(user_info["work3location (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work3location (S)"][0]) + " "
    if str(user_info["work3name (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work3name (S)"][0]) + " "
    if str(user_info["work3role (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work3role (S)"][0]) + " "
    if str(user_info["work4location (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work4location (S)"][0]) + " "
    if str(user_info["work4name (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work4name (S)"][0]) + " "
    if str(user_info["work4role (S)"][0]) != "nan":
        u_data = u_data + str(user_info["work4role (S)"][0]) + " "
    return u_data

In [191]:
def getUserArticleSim(article_data, u_data):
    docs = [article_data, u_data]
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    return cosine_similarities

In [193]:
users_userHashes = users["userHash (S)"]
articles_userHashes = articles["userHash (S)"]
valid_userHashes = list(set(users_userHashes) & set(articles_userHashes))

In [97]:
forgraph.head(n=5)

,index,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
1,19,-1748993256,http://hill.cm/EOrPLMZ,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""3 "" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:06:00 GMT"" }]","[ { ""S"" : ""NULL"" }]",White House refused to release photo of Trump ...,"[ { ""S"" : ""2407815"" }]"
2,20,-1748993256,http://hill.cm/X9HpS96,False,NaN,"[ { ""S"" : ""BREAKING: FBI officials were alert...","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/15/2018 8:05am"" }]",FBI was warned last year of school shooting th...,"[ { ""S"" : ""1101628624"" }]"
3,21,-1748993256,http://stanford.io/2ClZ68d,True,"Thu, 15 Feb 2018 14:32:01 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""44"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:32:01 GMT"" }]","[ { ""S"" : ""02/15/2018 8:45am"" }]","http://stanford.io/2ClZ68d</a> <br> <a class=""...","[ { ""S"" : ""46193696"" }]"
4,22,-1748993256,http://t.ted.com/gmdx0iv,False,NaN,"[ { ""S"" : ""Practical money advice from a self...","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 10:00am"" }]",6 simple tips for investment beginners,"[ { ""S"" : ""82931"" }]"
5,23,-1748993256,https://collegepulse.com/survey/5a85e6d386f8d3...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 9:40am"" }]",www.collegepulse.com</a>,"[ { ""S"" : ""1236633202"" }]"


In [134]:
temp = forgraph[forgraph["userHash (S)"] == 1832382593].reset_index()

In [135]:
temp.head(n=10)

,level_0,index,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
0,484,901,1832382593,http://b-gat.es/2CrhoVv,True,"Mon, 19 Feb 2018 22:16:44 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""18K"" }]","[ { ""S"" : ""Mon, 19 Feb 2018 22:16:44 GMT"" }]","[ { ""S"" : ""02/18/2018 8:40am"" }]",http://b-gat.es/2CrhoVv</a>,"[ { ""S"" : ""701632431"" }]"
1,485,902,1832382593,http://bit.ly/2Bdkm36,False,NaN,"[ { ""S"" : ""ItÛªs our big spring drop: 260 ne...","[ { ""S"" : ""5"" }]","[ { ""S"" : ""Thu, 08 Feb 2018 23:05:25 GMT"" }]","[ { ""S"" : ""feed_story_id_5a7cd484d17731b29939...",Up to 50% Off New Arrivals,"[ { ""S"" : ""-502270865"" }]"
2,486,903,1832382593,http://bit.ly/2E85ul7,True,"Mon, 19 Feb 2018 22:05:12 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""59"" }]","[ { ""S"" : ""Mon, 19 Feb 2018 22:05:12 GMT"" }]","[ { ""S"" : ""02/19/2018 7:05am"" }]",http://bit.ly/2E85ul7</a><br> Follow <a class=...,"[ { ""S"" : ""924107392"" }]"
3,487,904,1832382593,http://bit.ly/2EOZqle,True,"Sun, 18 Feb 2018 06:55:42 GMT","[ { ""S"" : ""¨_ø¨±ø¨±ø¨»ø¨»¨ø¨£ ...","[ { ""S"" : ""1.4K"" }]","[ { ""S"" : ""Sun, 18 Feb 2018 06:55:42 GMT"" }]","[ { ""S"" : ""02/17/2018 4:26am"" }]",¨_ø¨±ø¨±ø¨»ø¨»¨ø¨£ ¨µ¨À¨µ¨¡...,"[ { ""S"" : ""947973233"" }]"
4,488,906,1832382593,http://bit.ly/2FgTEGU,True,"Mon, 19 Feb 2018 22:19:45 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""2.1K"" }]","[ { ""S"" : ""Mon, 19 Feb 2018 22:19:45 GMT"" }]","[ { ""S"" : ""02/19/2018 12:13am"" }]","http://bit.ly/2FgTEGU</a></p><p> <a class=""_58...","[ { ""S"" : ""1036936401"" }]"
5,489,907,1832382593,http://bit.ly/2nR9LT8,True,"Sat, 17 Feb 2018 02:34:52 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""168"" }]","[ { ""S"" : ""Sat, 17 Feb 2018 02:34:52 GMT"" }]","[ { ""S"" : ""02/15/2018 10:39pm"" }]","http://bit.ly/2nR9LT8</a> <a class=""_58cn"" hre...","[ { ""S"" : ""537185968"" }]"
6,490,908,1832382593,http://bit.ly/2wvO91N,True,"Thu, 08 Feb 2018 19:27:45 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""96"" }]","[ { ""S"" : ""Thu, 08 Feb 2018 19:27:45 GMT"" }]","[ { ""S"" : ""02/07/2018 9:01am"" }]",http://bit.ly/2wvO91N</a>,"[ { ""S"" : ""-1950162835"" }]"
7,491,909,1832382593,http://cnn.it/2F85x1B,False,NaN,"[ { ""S"" : ""A former Playboy model who says sh...","[ { ""S"" : ""4.3K"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 16:40:12 GMT"" }]","[ { ""S"" : ""02/16/2018 6:00am"" }]",Ex-model details Trump system for covering up ...,"[ { ""S"" : ""66883"" }]"
8,492,911,1832382593,http://cnn.it/2nP55NA,True,"Thu, 08 Feb 2018 19:20:11 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""2.4K "" }]","[ { ""S"" : ""Thu, 08 Feb 2018 19:20:11 GMT"" }]","[ { ""S"" : ""02/08/2018 4:31am"" }]",http://cnn.it/2nP55NA</a>,"[ { ""S"" : ""66883"" }]"
9,493,912,1832382593,http://garammasalakitchen.com/punjabi-kadhi-ch...,True,"Mon, 19 Feb 2018 07:42:13 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""24"" }]","[ { ""S"" : ""Mon, 19 Feb 2018 07:42:09 GMT"" }]","[ { ""S"" : ""02/18/2018 9:12am"" }]",https://www.pinterest.com/garammasalakitchen/p...,"[ { ""S"" : ""744424215"" }]"


In [136]:
# get article data
comments = ast.literal_eval(temp["articleComments (L)"][7])[0].get("S")
title = temp["articleTitle (S)"][7]
article_data = comments + " " + title

In [137]:
article_data

'A former Playboy model who says she had an affair with <a class="profileLink" href="https://www.facebook.com/DonaldTrump/?fref=mentions" data-hovercard="/ajax/hovercard/page.php?id=153080620724&amp;extragetparams=%7B%22fref%22%3A%22mentions%22%7D" data-hovercard-prefer-more-content-show="1">Donald J. Trump</a> beginning in 2006 is going public with a description of Trump\'s alleged system for concealing affairs. Ex-model details Trump system for covering up affairs'

In [138]:
# tokenize
translator = str.maketrans('', '', string.punctuation)
article_data = article_data.lower().translate(translator) 
tokens = nltk.word_tokenize(article_data)

In [139]:
# remove stopwords
filtered_tokens = [w for w in tokens if not w in stopwords.words('english')]

In [140]:
# stem
stemmer = PorterStemmer()
stemmed_tokens = []
for item in filtered_tokens:
    stemmed_tokens.append(stemmer.stem(item))

In [141]:
Counter(stemmed_tokens)

Counter({'2006': 1,
         'affair': 3,
         'alleg': 1,
         'begin': 1,
         'classprofilelink': 1,
         'conceal': 1,
         'cover': 1,
         'datahovercardajaxhovercardpagephpid153080620724ampextragetparams7b22fref223a22mentions227d': 1,
         'datahovercardprefermorecontentshow1donald': 1,
         'descript': 1,
         'detail': 1,
         'exmodel': 1,
         'former': 1,
         'go': 1,
         'hrefhttpswwwfacebookcomdonaldtrumpfrefment': 1,
         'j': 1,
         'model': 1,
         'playboy': 1,
         'public': 1,
         'say': 1,
         'system': 2,
         'trump': 2,
         'trumpa': 1})

In [144]:
user_info = users[users["userHash (S)"] == 1832382593].reset_index()

In [164]:
# get user data
u_data = ""
if str(user_info["currentLocation (S)"][0]) != "nan":
    u_data = u_data + user_info["currentLocation (S)"][0] + " "
if str(user_info["education1location (S)"][0]) != "nan":
    u_data = u_data + user_info["education1location (S)"][0] + " "
if str(user_info["education1name (S)"][0]) != "nan":
    u_data = u_data + user_info["education1name (S)"][0] + " "
if str(user_info["education2location (S)"][0]) != "NaN":
    u_data = u_data + user_info["education2location (S)"][0] + " "
if str(user_info["education2name (S)"][0]) != "NaN":
    u_data = u_data + user_info["education2name (S)"][0] + " "
if str(user_info["education3location (S)"][0]) != "NaN":
    u_data = u_data + user_info["education3location (S)"][0] + " "
if str(user_info["education3name (S)"][0]) != "NaN":
    u_data = u_data + user_info["education3name (S)"][0] + " "
if str(user_info["gender (S)"][0]) != "NaN":
    u_data = u_data + user_info["gender (S)"][0] + " "
if str(user_info["hometown (S)"][0]) != "NaN":
    u_data = u_data + user_info["hometown (S)"][0] + " "
if str(user_info["language1 (S)"][0]) != "nan":
    u_data = u_data + str(user_info["language1 (S)"][0]) + " "
if str(user_info["language2 (S)"][0]) != "nan":
    u_data = u_data + str(user_info["language2 (S)"][0]) + " "
if str(user_info["language3 (S)"][0]) != "nan":
    u_data = u_data + str(user_info["language3 (S)"][0]) + " "
if str(user_info["otherLocation1 (S)"][0]) != "nan":
    u_data = u_data + str(user_info["otherLocation1 (S)"][0]) + " "
if str(user_info["otherLocation2 (S)"][0]) != "nan":
    u_data = u_data + str(user_info["otherLocation2 (S)"][0]) + " "
if str(user_info["work1location (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work1location (S)"][0]) + " "
if str(user_info["work1name (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work1name (S)"][0]) + " "
if str(user_info["work1role (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work1role (S)"][0]) + " "
if str(user_info["work2location (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work2location (S)"][0]) + " "
if str(user_info["work2name (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work2name (S)"][0]) + " "
if str(user_info["work2role (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work2role (S)"][0]) + " "
if str(user_info["work3location (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work3location (S)"][0]) + " "
if str(user_info["work3name (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work3name (S)"][0]) + " "
if str(user_info["work3role (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work3role (S)"][0]) + " "
if str(user_info["work4location (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work4location (S)"][0]) + " "
if str(user_info["work4name (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work4name (S)"][0]) + " "
if str(user_info["work4role (S)"][0]) != "nan":
    u_data = u_data + str(user_info["work4role (S)"][0]) + " "

In [177]:
u_data = getUserData(1832382593)

In [179]:
Counter(stem(removeStopwords(tokenize(u_data))))

Counter({'anna': 1,
         'chennai': 2,
         'colleg': 1,
         'engin': 1,
         'femal': 1,
         'guindi': 1,
         'india': 4,
         'ladi': 1,
         'mctm': 1,
         'ohio': 1,
         'sammamish': 1,
         'school': 1,
         'state': 1,
         'univers': 2,
         'wa': 1})

In [180]:
article_data

'a former playboy model who says she had an affair with a classprofilelink hrefhttpswwwfacebookcomdonaldtrumpfrefmentions datahovercardajaxhovercardpagephpid153080620724ampextragetparams7b22fref223a22mentions227d datahovercardprefermorecontentshow1donald j trumpa beginning in 2006 is going public with a description of trumps alleged system for concealing affairs exmodel details trump system for covering up affairs'

In [181]:
u_data

'Sammamish, WA India College of Engineering, Guindy, Anna University India The Ohio State University Chennai, India Lady M.CT.M. School Female Chennai, India '

In [182]:
docs = [article_data, u_data]
tfidf = TfidfVectorizer(stop_words='english')
output = tfidf.fit_transform(docs)
cosine_similarities = linear_kernel(output[0:1], output).flatten()

In [61]:
tfidf = TfidfVectorizer(stop_words='english')

In [183]:
output = tfidf.fit_transform(docs)

In [186]:
output[0:1]

<1x37 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [188]:
cosine_similarities = linear_kernel(output[0:1], output).flatten()

In [189]:
cosine_similarities

array([ 1.,  0.])